In [25]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [26]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

verbose = True
# verbose = False

In [27]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df.head())

len(df): 44
  date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12 sym_freq_11 sym_freq_10 sym_freq_9 sym_freq_8 sym_freq_7             sym_freq_6                     sym_freq_5                     sym_freq_4                     sym_freq_3 sym_freq_2
0        2023-04-14                 120      [30, 60, 120]          []          []          []          []          []          []         []    ['SHV']         []        ['ELF', 'FTSM']  ['AMPH', 'NVDA', 'NVO', 'P...  ['AU', 'COTY', 'GE', 'LNTH...  ['ACLS', 'ANET', 'AZN', 'B...         []
1        2023-04-14                 120  [15, 30, 60, 120]          []          []          []          []          []          []         []    ['SHV']         []  ['AU', 'ELF', 'FTSM']  ['AMPH', 'NVDA', 'NVO', 'P...  ['COTY', 'GE', 'LNTH', 'MA...               ['ACLS', 'ANET']         []
2        2023-04-13                 120      [30, 60, 120]          []          []          []          []         

In [28]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  _date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == _date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if _date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = _date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-04-14    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['SHV']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : []
5                   [15, 30, 60, 120]    sym_freq_8 : ['SHV']

6     2023-04-13    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['SHV']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : ['SHV']
11                  [15, 30, 60, 120]    sym_freq_8 : []

12    2023-04-12    [30, 60, 120]        sym

In [29]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [30]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-04-14 [30, 60, 120] sym_freq_12 []
1 2023-04-14 [30, 60, 120] sym_freq_9 []
2 2023-04-14 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-14', [30, 60, 120], 'sym_freq_8', ['SHV']]

3 2023-04-14 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-14 [15, 30, 60, 120] sym_freq_9 []
5 2023-04-14 [15, 30, 60, 120] sym_freq_8 ['SHV']
6 2023-04-13 [30, 60, 120] sym_freq_12 []
7 2023-04-13 [30, 60, 120] sym_freq_9 []
8 2023-04-13 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-13', [30, 60, 120], 'sym_freq_8', ['SHV']]

9 2023-04-13 [15, 30, 60, 120] sym_freq_12 []
10 2023-04-13 [15, 30, 60, 120] sym_freq_9 ['SHV']
11 2023-04-13 [15, 30, 60, 120] sym_freq_8 []
12 2023-04-12 [30, 60, 120] sym_freq_12 []
13 2023-04-12 [30, 60, 120] sym_freq_9 []
14 2023-04-12 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-12', [30, 60, 120], 'sym_freq_8', ['SHV']]

15 2023-04-12 [15, 30, 60, 120] sym_freq_12 []
16 2023-04-12 [15,

In [31]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates(keep='last')
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 27
df_model_top_picks:
          date  days_lookback   syms_freq          symbols
0   2023-04-14  [30, 60, 120]  sym_freq_8          ['SHV']
1   2023-04-13  [30, 60, 120]  sym_freq_8          ['SHV']
2   2023-04-12  [30, 60, 120]  sym_freq_8          ['SHV']
3   2023-04-11  [30, 60, 120]  sym_freq_8          ['SHV']
4   2023-04-10  [30, 60, 120]  sym_freq_8          ['SHV']
5   2023-04-06  [30, 60, 120]  sym_freq_8          ['SHV']
6   2023-04-05  [30, 60, 120]  sym_freq_9          ['SHV']
7   2023-04-04  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
8   2023-04-03  [30, 60, 120]  sym_freq_8   ['GE', 'NVDA']
9   2023-03-31  [30, 60, 120]  sym_freq_8         ['NVDA']
10  2023-03-30  [30, 60, 120]  sym_freq_9          ['SHV']
11  2023-03-30  [30, 60, 120]  sym_freq_8         ['NVDA']
12  2023-03-29  [30, 60, 120]  sym_freq_9          ['SHV']
13  2023-03-28  [30, 60, 120]  sym_freq_9          ['SHV']
14  2023-03-27  [30, 60, 120]  sym_freq_9          ['SHV']
15  2023

In [32]:
def get_next_date(df_close, date):
    """Get the next date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + 1  # get index position of next date

        if idx_next_date <= (len_df_close - 1):  # next date's index is within bound of df_close
            next_date = idx_dates[idx_dates.get_loc(date) + 1]  # get the next date
            return next_date  
        else:  # idx_next_date is out-of_bounds df_close
            print(f"{date} is the last date in df_close. No data for next day")
            return None    
    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [33]:
def get_symbols_close(my_date, df_close):
  # get closing price for symbols for my_date, i.e. a row of df_close for my_date
  # return symbols' closing price as a series
  if my_date in df_close.index:  
    iloc_my_date = df_close.index.get_loc(my_date)
    print(f'len(df_close): {len(df_close)}, my_date: {my_date}, iloc_my_date: {iloc_my_date}')      
    symbols_close = df_close.iloc[iloc_my_date]    
  else:
    msg_stop = f'ERROR: my_date: {my_date} is not a date in df_close.index\ndf_close.index:\n{df_close.index}'
    raise SystemExit(msg_stop)
  return symbols_close

In [34]:
def get_portf_buy(df_close, date, str_symbols, portf_target, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
     # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    sym_cnt = len(l_syms)  # number of symbols
    amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
    ar_sym_share = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
    ar_sym_amt = ar_price * ar_sym_share  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_sym_amt)  # total actual portfolio value
    if verbose:
        # print(date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value)    
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_sym_share = }, {ar_sym_amt = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value

In [35]:
def get_SPY_buy(df_close, date, portf_target, symbol="SPY", verbose=False):   
    import numpy as np
    price = df_close.loc[date][symbol]    
    share = np.floor(portf_target / price)  # array of shares for each symbol
    value = price * share  # array of actual dollar amount invested in each symbol
    if verbose:
        # print(date, symbol, price, share, value)
        print(f'{date = }, {symbol = }, {price = }, {share = }, {value = }')                        
        print(f'{date} {symbol} {value = }')
    return date, price, share, value

In [36]:
def get_NYSE_date_n(date, n, dates_NYSE, verbose=False):

  if date in dates_NYSE:
    idx_last = len(dates_NYSE) - 1  # last index of dates_NYSE
    idx_date = dates_NYSE.get_loc(date)  # index of date
    idx_date_n = idx_date + n  # index of date_n
    if verbose:
      print(f"date: {date} is in dates_NYSE,  date's position in dates_NYSE is: {idx_date} of {idx_last},  n: {n},  idx_date_n: {idx_date_n},")
    if 0 <= idx_date_n <= idx_last:  # idx_date_n is within bounds of dates_NYSE
      date_n = dates_NYSE[idx_date_n]
      if verbose:      
        print(f'idx_date_n: {idx_date_n} is wiithin bounds of dates_NYSE ("0 to {idx_last}), date_n: {date_n}"\n')
    else:  # idx_date_n is out-of-bounds of dates_NYSE
      date_n = None   
      if verbose:  
        print(f'idx_date_n: {idx_date_n} is out-of-bounds of dates_NYSE ("0 to {idx_last})"\n')
  else:  # date not in dates_NYSE
      date_n = None  
      if verbose:  
        print(f'date: {date} is not in dates_NYSE\n')

  return date_n  

In [37]:
def get_next_n_date(df_close, date, n=1):

    """Get the next n(default=1) date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + n  # get index position of next date
        print(f'n: {n},  idx_next_date: {idx_next_date}')

        if idx_next_date >= 0:  # if idx_nex_date is negative, stop index loop back from index[0] to index[-1]    
            if idx_next_date >= (len_df_close - n):  # next date's index is within bound of df_close
                next_date = idx_dates[idx_dates.get_loc(date) + n]  # get the next date
                return next_date
        
              
            else:  # idx_next_date is out-of_bounds df_close
                print(f"{date} is the last date in df_close. No data for next {n} day")
                return None    
        
        
        else:  # idx_next_date is negative, no data in df_close
            # no data from df_close
            print(f'idx_next_date: {idx_next_date} is negative, no data for {n} days from {date} in df_close')

    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [38]:
def get_portf_value(my_dates, my_symbols, df_close, n):
  
  z_date_syms = zip(my_dates, my_symbols)

  idx_close = df_close.index.strftime('%Y-%m-%d')

  # n = 1  # n=1 is the buy date
  # n = 4  # n=4 is the sell date

  date_buy = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date_buy
  value_portf = [] # list of porfolio value on date_buy
  shares_SPY = []  # list of shares of SPY brought on date_buy
  value_SPY = []  # list of value of SPY shares on date_buy 
  for date, syms in z_date_syms:
    next_n_date = get_NYSE_date_n(date, n, dates_NYSE_2013_2023, verbose=verbose)

    if next_n_date in idx_close:
      close_n_date = next_n_date
      print(f'date: {date},  n: {n},  next_n_date: {next_n_date} is in df_close')
      print(f'picks for {date}: {syms}')      
    else:
      close_n_date = None
      print(f'date: {date},  n: {n},  next_n_date: {next_n_date} is not in df_close')
      print(f'picks for {date}: {syms}')        

    if close_n_date is None:
      p_date = None
      p_l_syms = None
      p_ar_price = None
      p_ar_sym_share = None
      p_ar_sym_amt = None
      p_portf_value = None  # data not available in df_close
      
      s_date = None
      s_price = None
      s_share = None
      s_value = None  # data not available in df_close

      date_buy.append(p_date)
      shares_syms.append(p_l_syms)
      value_portf.append(p_portf_value)
      shares_SPY.append(s_share)
      value_SPY.append(s_value)    
      
      print(f"No data for next_n_date {next_n_date}, pick's portf value = None")
      print(f'No data for next_n_date {next_n_date}, SPY portf value =    None')

    else:    
      p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
          get_portf_buy(df_close, next_n_date, syms, portf_target=1000, verbose=verbose)
      s_date, s_price, s_share, s_value = \
          get_SPY_buy(df_close, next_n_date, portf_target=1000, verbose=verbose)
              
      date_buy.append(p_date)
      shares_syms.append(p_ar_sym_share)
      value_portf.append(p_portf_value)
      shares_SPY.append(s_share)
      value_SPY.append(s_value)

      print(f"next_n_date pick's portf value = {p_portf_value}")
      print(f'next_n_date SPY portf value =    {s_value}')

      print('='*20, '\n')

  return date_buy, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [39]:
import pandas_market_calendars as mcal
# import datetime
nyse = mcal.get_calendar('NYSE')
dates_NYSE_2013_2023 = nyse.valid_days(start_date='2013-01-01', end_date='2023-12-31').strftime('%Y-%m-%d')
print(f'len(dates_NYSE_2013_2023): {len(dates_NYSE_2013_2023)}')
print(f'dates_NYSE_2013_2023[0]:  {dates_NYSE_2013_2023[0]}, dates_NYSE_2013_2023[-1]: {dates_NYSE_2013_2023[-1]}')
print(f'{dates_NYSE_2013_2023}') 

len(dates_NYSE_2013_2023): 2768
dates_NYSE_2013_2023[0]:  2013-01-02, dates_NYSE_2013_2023[-1]: 2023-12-29
Index(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11', '2013-01-14', '2013-01-15',
       ...
       '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29'], dtype='object', length=2768)


In [40]:
df_mp = df_model_top_picks.copy()
df_close = pickle_load(path_data_dump, 'df_close_clean')

In [41]:
dates = df_mp.date
symbols = df_mp.symbols

In [42]:
date_buy, shares_syms, value_portf, shares_SPY, value_SPY = get_portf_value(dates, symbols, df_close, n=1)

date: 2023-04-14 is in dates_NYSE,  date's position in dates_NYSE is: 2588 of 2767,  n: 1,  idx_date_n: 2589,
idx_date_n: 2589 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-17"

date: 2023-04-14,  n: 1,  next_n_date: 2023-04-17 is not in df_close
picks for 2023-04-14: ['SHV']
No data for next_n_date 2023-04-17, pick's portf value = None
No data for next_n_date 2023-04-17, SPY portf value =    None
date: 2023-04-13 is in dates_NYSE,  date's position in dates_NYSE is: 2587 of 2767,  n: 1,  idx_date_n: 2588,
idx_date_n: 2588 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-14"

date: 2023-04-13,  n: 1,  next_n_date: 2023-04-14 is in df_close
picks for 2023-04-13: ['SHV']
date = '2023-04-14', l_syms = ['SHV'], ar_price = array([110.26000214]), ar_sym_share = array([9.]), ar_sym_amt = array([992.34001923]), portf_value = 992.3400192260742
2023-04-14 portf_value = 992.3400192260742
date = '2023-04-14', symbol = 'SPY', price = 412.4599914550781, share = 2.0, value

In [43]:
print(f'len(df_model_top_picks): {len(df_model_top_picks)}\n')
print(f'date_buy, {len(date_buy) = }, {type(date_buy) = }:\n{date_buy}\n')
print(f'shares_syms, {len(shares_syms) = }, {type(shares_syms) = }:\n{shares_syms}\n')
print(f'value_portf, {len(value_portf) = }, {type(value_portf) = }:\n{value_portf}\n')
print(f'shares_SPY, {len(shares_SPY) = }, {type(shares_SPY) = }:\n{shares_SPY}\n')
print(f'value_SPY, {len(value_SPY) = }, {type(value_SPY) = }:\n{value_SPY}\n')

len(df_model_top_picks): 27

date_buy, len(date_buy) = 27, type(date_buy) = <class 'list'>:
[None, '2023-04-14', '2023-04-13', '2023-04-12', '2023-04-11', '2023-04-10', '2023-04-06', '2023-04-05', '2023-04-04', '2023-04-03', '2023-03-31', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-28', '2023-03-27', '2023-03-24', '2023-03-23', '2023-03-23', '2023-03-22', '2023-03-22', '2023-03-21', '2023-03-20', '2023-03-20', '2023-03-17', '2023-03-16']

shares_syms, len(shares_syms) = 27, type(shares_syms) = <class 'list'>:
[None, array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([1., 4.]), array([5., 1.]), array([3.]), array([9.]), array([3.]), array([9.]), array([9.]), array([9.]), array([10.]), array([1., 4.]), array([9.]), array([9.]), array([16.]), array([9.]), array([11.]), array([8., 4.]), array([8., 4.]), array([11.]), array([8., 4.]), array([8., 4.])]

value_portf, len(value_portf) = 27, type(value_portf) = <class 'list'>:
[None, 992.340

In [44]:
import numpy as np
# from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)

idx_close = df_close.index.strftime('%Y-%m-%d')

n = 1  # n=1 is the buy date
# n = 4  # n=4 is the sell date

date_buy = []  # buy date of portfolio
shares_syms = []  # lists of shares of each symbol brought on date_buy
value_portf = [] # list of porfolio value on date_buy
shares_SPY = []  # list of shares of SPY brought on date_buy
value_SPY = []  # list of value of SPY shares on date_buy 
for date, syms in z_date_syms:
    next_n_date = get_NYSE_date_n(date, n, dates_NYSE_2013_2023, verbose=verbose)

    if next_n_date in idx_close:
        close_n_date = next_n_date
        print(f'date: {date},  n: {n},  next_n_date: {next_n_date} is in df_close')
        print(f'picks for {date}: {syms}')      
    else:
        close_n_date = None
        print(f'date: {date},  n: {n},  next_n_date: {next_n_date} is not in df_close')
        print(f'picks for {date}: {syms}')        

    if close_n_date is None:
        p_date = None
        p_l_syms = None
        p_ar_price = None
        p_ar_sym_share = None
        p_ar_sym_amt = None
        p_portf_value = None  # data not available in df_close
        
        s_date = None
        s_price = None
        s_share = None
        s_value = None  # data not available in df_close

        date_buy.append(p_date)
        shares_syms.append(p_l_syms)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)    
        
        print(f"No data for next_n_date {next_n_date}, pick's portf value = None")
        print(f'No data for next_n_date {next_n_date}, SPY portf value =    None')

    else:    
        p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
            get_portf_buy(df_close, next_n_date, syms, portf_target=1000, verbose=verbose)
        s_date, s_price, s_share, s_value = \
            get_SPY_buy(df_close, next_n_date, portf_target=1000, verbose=verbose)
                
        date_buy.append(p_date)
        shares_syms.append(p_ar_sym_share)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)

        print(f"next_n_date pick's portf value = {p_portf_value}")
        print(f'next_n_date SPY portf value =    {s_value}')

    print('='*20, '\n')       
    

date: 2023-04-14 is in dates_NYSE,  date's position in dates_NYSE is: 2588 of 2767,  n: 1,  idx_date_n: 2589,
idx_date_n: 2589 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-17"

date: 2023-04-14,  n: 1,  next_n_date: 2023-04-17 is not in df_close
picks for 2023-04-14: ['SHV']
No data for next_n_date 2023-04-17, pick's portf value = None
No data for next_n_date 2023-04-17, SPY portf value =    None

date: 2023-04-13 is in dates_NYSE,  date's position in dates_NYSE is: 2587 of 2767,  n: 1,  idx_date_n: 2588,
idx_date_n: 2588 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-04-14"

date: 2023-04-13,  n: 1,  next_n_date: 2023-04-14 is in df_close
picks for 2023-04-13: ['SHV']
date = '2023-04-14', l_syms = ['SHV'], ar_price = array([110.26000214]), ar_sym_share = array([9.]), ar_sym_amt = array([992.34001923]), portf_value = 992.3400192260742
2023-04-14 portf_value = 992.3400192260742
date = '2023-04-14', symbol = 'SPY', price = 412.4599914550781, share = 2.0, valu

In [45]:
print(f'len(df_model_top_picks: {len(df_model_top_picks)}\n')
print(f'date_buy, {len(date_buy) = }, {type(date_buy) = }:\n{date_buy}\n')
print(f'shares_syms, {len(shares_syms) = }, {type(shares_syms) = }:\n{shares_syms}\n')
print(f'value_portf, {len(value_portf) = }, {type(value_portf) = }:\n{value_portf}\n')
print(f'shares_SPY, {len(shares_SPY) = }, {type(shares_SPY) = }:\n{shares_SPY}\n')
print(f'value_SPY, {len(value_SPY) = }, {type(value_SPY) = }:\n{value_SPY}\n')

len(df_model_top_picks: 27

date_buy, len(date_buy) = 27, type(date_buy) = <class 'list'>:
[None, '2023-04-14', '2023-04-13', '2023-04-12', '2023-04-11', '2023-04-10', '2023-04-06', '2023-04-05', '2023-04-04', '2023-04-03', '2023-03-31', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-28', '2023-03-27', '2023-03-24', '2023-03-23', '2023-03-23', '2023-03-22', '2023-03-22', '2023-03-21', '2023-03-20', '2023-03-20', '2023-03-17', '2023-03-16']

shares_syms, len(shares_syms) = 27, type(shares_syms) = <class 'list'>:
[None, array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([9.]), array([1., 4.]), array([5., 1.]), array([3.]), array([9.]), array([3.]), array([9.]), array([9.]), array([9.]), array([10.]), array([1., 4.]), array([9.]), array([9.]), array([16.]), array([9.]), array([11.]), array([8., 4.]), array([8., 4.]), array([11.]), array([8., 4.]), array([8., 4.])]

value_portf, len(value_portf) = 27, type(value_portf) = <class 'list'>:
[None, 992.3400

In [46]:
# df_model_picks_perf = df_model_top_picks.copy()
_df = df_model_top_picks.copy()
_df.head()

,date,days_lookback,syms_freq,symbols
0,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV']
1,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV']
2,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV']
3,2023-04-11,"[30, 60, 120]",sym_freq_8,['SHV']
4,2023-04-10,"[30, 60, 120]",sym_freq_8,['SHV']


In [47]:
_df['date_buy'] = date_buy
_df['sh_syms'] = shares_syms
_df['$_portf_buy'] = value_portf
_df['sh_SPY'] = shares_SPY
_df['$_SPY_buy'] = value_SPY
_df.head()

,date,days_lookback,syms_freq,symbols,date_buy,sh_syms,$_portf_buy,sh_SPY,$_SPY_buy
0,2023-04-14,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,NaN,NaN
1,2023-04-13,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-14,[9.0],992.340019,2.0,824.919983
2,2023-04-12,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-13,[9.0],992.519989,2.0,826.940002
3,2023-04-11,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-12,[9.0],991.980011,2.0,816.099976
4,2023-04-10,"[30, 60, 120]",sym_freq_8,['SHV'],2023-04-11,[9.0],991.889992,2.0,819.440002


In [48]:
my_dates = ['2013-01-01', '2013-01-02', '2013-01-12', '2023-12-29', '2023-12-30', '2023-04-13']
n = 1
dates_NYSE = dates_NYSE_2013_2023

idx_close = df_close.index.strftime('%Y-%m-%d')

for date in my_dates:
    date_n = get_NYSE_date_n(date, n, dates_NYSE, verbose=False)
    print(f'date: {date},  n: {n},  date_n: {date_n}')
    if date_n in idx_close:
        print(f'date_n: {date_n} is in df_close\n')
    else:
        print(f'date_n: {date_n} is not in df_close\n')


date: 2013-01-01,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2013-01-02,  n: 1,  date_n: 2013-01-03
date_n: 2013-01-03 is not in df_close

date: 2013-01-12,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-12-29,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-12-30,  n: 1,  date_n: None
date_n: None is not in df_close

date: 2023-04-13,  n: 1,  date_n: 2023-04-14
date_n: 2023-04-14 is in df_close

